In [43]:
from openai import OpenAI
import datasets
from tqdm import tqdm

In [46]:
client = OpenAI()
buggy_data = datasets.load_dataset("lrhammond/buggy-apps", split="test")
flags = {}

In [18]:
i = 0

for i in tqdm(range(len(buggy_data))):

    datum = buggy_data[i]
    if i in flags:
        continue

    response = client.moderations.create(
        model="omni-moderation-latest",
        input=datum['question'],
        )
    
    if response.results[0].flagged:
        
        categories = dict(response.results[0].categories)
        flags[i] = ",".join([k for k in categories if categories[k]])
    
    else:

        flags[i] = ""

 33%|███▎      | 160/488 [00:49<01:35,  3.42it/s]

In [29]:
for k in flags:
    if flags[k] != "":
        print(f"{k}: {flags[k]}")

34: violence
46: violence
104: violence
121: violence
171: violence
181: violence
189: violence
243: violence
294: violence
383: violence
454: violence
489: violence
491: violence
613: violence
674: violence
784: violence
948: violence
1065: violence
1080: violence
1190: violence
1269: violence
1272: violence
1344: violence
1451: violence
1470: violence
1520: violence
1525: violence
1753: violence
1789: violence
1803: violence
1807: violence
1982: violence
2010: violence
2078: violence,illicit/violent
2136: violence
2213: violence
2234: violence
2304: violence
2330: violence
2434: violence
2488: violence
2623: violence
2669: violence
2736: harassment
2855: violence
2882: violence
3078: violence
3120: violence
3271: violence
3363: violence
3426: violence
3432: violence
3449: violence
3453: violence
3629: violence
3759: violence
3769: violence
3776: illicit
3810: violence
3831: violence
3855: violence
3889: violence
4110: self_harm,violence,self-harm
4118: violence
4353: violence


In [37]:
updated_buggy_data = buggy_data.add_column("flags", flags.values())

In [40]:
updated_buggy_data[3759]

{'original_split': 'train',
 'original_problem_id': 3004,
 'difficulty': 'introductory',
 'question': "It's Friday night, and Chuck is bored. He's already run 1,000 miles, stopping only to eat a family sized bag of Heatwave Doritos and a large fistful of M&Ms. He just can't stop thinking about kicking something! \n\nThere is only one thing for it, Chuck heads down to his local MMA gym and immediately challenges every fighter there to get in the cage and try and take him down... AT THE SAME TIME! \n\nYou are provided an array of strings that represent the cage and Chuck's opponents. Your task, in traditional Chuck style, is to take their heads off!! Throw punches, kicks, headbutts (or more likely - regex or iteration...) but whatever you do, remove their heads. Return the same array of strings, but with the heads ('O') removed and replaced with a space (' ').\n\nIf the provided array is empty, or is an empty string, return 'Gym is empty'. If you are given an array of numbers, return 'Th

In [45]:
updated_buggy_data.to_parquet("test.parquet")

Creating parquet from Arrow format:   0%|          | 0/5 [00:00<?, ?ba/s]

11597804

In [ ]:
updated_buggy_data.push_to_hub(
    "lrhammond/buggy-apps",
    commit_message="Add flags",
    )